In [131]:
#!pip install bayesian-optimization  # for google collab
#!pip install git+https://github.com/slremy/netsapi --user --upgrade


In [132]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import UtilityFunction
from netsapi.challenge import *
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import mlab
from matplotlib import gridspec
%matplotlib inline
from sys import exit, exc_info, argv
from multiprocessing import Pool, current_process
import random as rand
import json
import requests
import numpy as np
import pandas as pd
import statistics
from IPython.display import clear_output

from contextlib import contextmanager
import sys, os
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import math

def newHoleEnv():
    return ChallengeProveEnvironment()
    #return ChallengeSeqDecEnvironment() #Initialise a New Challenge Environment to post entire policy

print("done")

done


# Utility : Neural net

In [133]:

# action val in
x_start = 0.0
x_end = 1.0

old_action = np.array([0.0 , 0.0])
weight = np.array([0.0 , 0.0]) # todo init rand


def nn_sum(w0, w1, x1): # affine
    return w0 + w1*x1
def nn_mul(a,b):
    return a*b
def nn_Modified_sigmoid(x):
    return nn_sigmoid(x)/2 + 0.5
def nn_sigmoid(x):
    return (math.exp( 2.0*x ) - 1.0 ) / (math.exp( 2.0*x ) + 1.0)
def nn_porte(x, mn, mx):
    return max(mn , min(x,mx) )

def nn(action):
    global old_action
    a1 = nn_sum(weight[0], weight[1], old_action)
    a2 = nn_sum(weight[0], weight[1], action)
    a3 = nn_mul(a1,a2)
    a3 = nn_Modified_sigmoid(a3)
    a4 = nn_porte(a3, x_start, x_end)
    
    
    
    
def nn_balance(w0, a1, a2):
    return w0*a1 + (1.0-w0)*a2
def nn_simple_sum(a,b):
    return a+b


In [134]:
policyID_queue = [] # Example [0,1,2]
target_out = [] # example [1]


# init
policies = []
detailed_rewards = []
UsedPolicy = 0

def target1(**kwargs):
    global policyID_queue, detailed_rewards
    reward = 0.0
    if len( policyID_queue ) > 0 :
        policyID = policyID_queue.pop(0)
        detailed_reward = detailed_rewards[   policyID   ]
        
        for key in target_out:
            #print("detailed_reward: " , detailed_reward[key])
            reward += detailed_reward[key]
    else: # do it online   
        raise SyntaxError('mini BOs should never query online ')
        #Raises:
        #      Runtime error
        #     mini BOs should never query online 
    #print("target1 " , reward)    
    return reward  #  /90.0  


def target(**kwargs):  
    modeLista = False
    valueLen = 0
    for key, value in kwargs.items(): 
        #print ("target:: ",key," == ",value) 
        if type(value) is np.ndarray:
            modeLista = True
            valueLen = len(value)
        break
    if modeLista:
        result = []
        for _ in range(valueLen):
            reward = target1()
            result.append( reward )
        return result
    else:
        return target1()


In [135]:
def train_BO_hm(policy_trainSet):
    global policyID_queue,target_out
    
    bos = [] 
    bos_target_out = []
    
    #TODO loop
    bos.append( BayesianOptimization(target, {'1x': (x_start, x_end), '1y': (x_start, x_end)})   )
    bos_target_out.append( [1] )
    
    for bo,aBO_target_out in zip(bos, bos_target_out):
        policyID_queue = policy_trainSet[:]
        target_out = aBO_target_out
        #bo.maximize(init_points=5, n_iter=0, acq='ucb', kappa=10)
        for apolicyID in policy_trainSet:
            #print("queue: ", policyID_queue) it don t change here
            #print("apolicyID ",apolicyID)
            apolicy = policies[apolicyID]
            #print("apolicy ",apolicy)
            params = {}
            for key in bo.space.keys:
                params[key]=apolicy[key]
            #print("probe for param: ", params )    
            bo.probe(params=params,lazy=True)    
            bo.maximize(init_points=0, n_iter=0, acq='ucb')
        print("now queue should be empty: ", policyID_queue)
    return bos


def train():
    #leave 1 out for the balance weights and MSE
    for LeftPolicy in range(1): #todo put it back range(UsedPolicy):
        ### set ids ...
        print("UsedPolicy ", UsedPolicy)
        policy_trainSet = list( range(UsedPolicy) )
        policy_trainSet.pop(LeftPolicy)
        print("policy_trainSet ", policy_trainSet)
        
        ## train BO hm
        train_BO_hm(policy_trainSet)
         
        ## train weights
        ## MSE
    
    # keep the best weights and BO hm regarding the MSE
    # add the left policy to the hm
    # 1 BO 10 dim
    # 1 suggestion

# main

In [136]:

env = newHoleEnv()

#hyper param
x_start = 0.0
x_end = 1.0
normalisation = 90.0 # reward normalisation
initial_rand_policy = 6 # 5 train + 1 test


#code
for _ in range(initial_rand_policy):
        policy = {'1x':rand.random(),'1y':rand.random(),'2x':rand.random(),'2y':rand.random(),'3x':rand.random(),'3y':rand.random(),'4x':rand.random(),'4y':rand.random(),'5x':rand.random(),'5y':rand.random()}
        detailed_reward= {1:0, 2:0, 3:0, 4:0, 5:0}
        env.reset()
        #reward_AllPolicy = env.evaluatePolicy(policy)
        for i in range (1,6):
            nextstate, reward, done, _ = env.evaluateAction([ policy[str(i)+'x'] , policy[str(i)+'y']  ] )
            detailed_reward[i] = reward/ normalisation
            
        policies.append(policy)
        detailed_rewards.append(detailed_reward)
UsedPolicy = initial_rand_policy          

#print(policies)
#print(detailed_rewards)
train()           
         


105  Evaluations Remaining
104  Evaluations Remaining
103  Evaluations Remaining
102  Evaluations Remaining
101  Evaluations Remaining
100  Evaluations Remaining
99  Evaluations Remaining
98  Evaluations Remaining
97  Evaluations Remaining
96  Evaluations Remaining
95  Evaluations Remaining
94  Evaluations Remaining
93  Evaluations Remaining
92  Evaluations Remaining
91  Evaluations Remaining
90  Evaluations Remaining
89  Evaluations Remaining
88  Evaluations Remaining
87  Evaluations Remaining
86  Evaluations Remaining
UsedPolicy  4
policy_trainSet  [1, 2, 3]
|   iter    |  target   |    1x     |    1y     |
-------------------------------------------------
target::  1x  ==  0.29547971537356377
detailed_reward:  -0.32063986195114524
target1  -0.32063986195114524
|  1        | -0.3206   |  0.2955   |  0.45     |
|   iter    |  target   |    1x     |    1y     |
-------------------------------------------------
target::  1x  ==  0.34256492512506664
detailed_reward:  -0.3871706430146031
